In [137]:
from matplotlib import pyplot as plt
from matplotlib import dates as mpl_dates
import yfinance as yf
import pandas as pd
import numpy as np
from helpers import Trade
import os

plt.style.use('Solarize_Light2')

In [138]:
symbols_list = [
    "GC=F",
    "NQ=F",
    "YM=F",
]

end_date = '2024-10-08'

start_date = pd.to_datetime(end_date)-pd.DateOffset(364 * 2)

if os.path.exists("global-data.csv"):
    df = pd.read_csv("global-data.csv")
else:
    df = yf.download(tickers=symbols_list, start=start_date, end=end_date, interval="1h").stack(future_stack=True)
    df.drop(columns=['Volume'], inplace=True)
    # df = yf.download(tickers=symbols_list, start=start_date, end=end_date, interval="1h").stack(future_stack=True)
    df.to_csv("global-data.csv")

df

# df.index = pd.to_datetime(df.index, utc=True)

# df.index.names = ['date', 'ticker']

# df.columns = df.columns.str.lower()

# display(df.shape)
# display(df.head(3))

,Datetime,Ticker,Adj Close,Close,High,Low,Open
0,2022-10-11 04:00:00+00:00,GC=F,1670.000000,1670.000000,1671.900024,1668.50,1670.699951
1,2022-10-11 04:00:00+00:00,NQ=F,10926.250000,10926.250000,10947.250000,10913.25,10936.750000
2,2022-10-11 04:00:00+00:00,YM=F,29091.000000,29091.000000,29132.000000,29068.00,29105.000000
3,2022-10-11 05:00:00+00:00,GC=F,1673.099976,1673.099976,1673.500000,1669.50,1669.900024
4,2022-10-11 05:00:00+00:00,NQ=F,10927.500000,10927.500000,10940.500000,10916.75,10926.500000
...,...,...,...,...,...,...,...
34336,2024-10-08 02:00:00+00:00,NQ=F,19999.500000,19999.500000,20012.000000,19955.50,19979.750000
34337,2024-10-08 02:00:00+00:00,YM=F,42230.000000,42230.000000,42242.000000,42193.00,42233.000000
34338,2024-10-08 03:00:00+00:00,GC=F,2661.800049,2661.800049,2664.800049,2654.50,2654.800049
34339,2024-10-08 03:00:00+00:00,NQ=F,19977.750000,19977.750000,20012.000000,19970.00,19998.250000


In [139]:
df.index

RangeIndex(start=0, stop=34341, step=1)

In [140]:
# symbols_list = [
#     "GC=F",
#     "NQ=F",
#     "YM=F",
# ]

# end_date = '2024-10-08'

# start_date = pd.to_datetime(end_date)-pd.DateOffset(364 * 2)

# df = yf.download(tickers=symbols_list, start=start_date, end=end_date, interval="1h").stack(future_stack=True)

# df.index.names = ['datetime', 'ticker']

# df.columns = df.columns.str.lower()

# df